In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('Training_Data.csv')

In [ ]:
df.head()

,Company Name,Business Description,Industry Classification Tag
0,"ADSOUTH PARTNERS, INC.","Adsouth Partners, Inc. provides advertising ag...",Advertising
1,"Artec Global Media, Inc.","Artec Global Media, Inc., formerly Artec Consu...",Advertising
2,Betawave Corp.,Betawave Corporation provides online marketing...,Advertising
3,BOSTON OMAHA Corp,Boston Omaha Corporation is engaged in the bus...,Advertising
4,Bright Mountain Media Inc,"Bright Mountain Media, Inc. is a digital media...",Advertising


In [ ]:
df.shape

(6045, 3)

In [ ]:
# Drop the Company Name Column as it is not relevant in determining the Industry Classification Tag
df = df.drop('Company Name',axis = 1)

Find the number of null values

In [ ]:
df.isnull().sum()

Business Description           23
Industry Classification Tag     0
dtype: int64

Dropping all NAN values

In [ ]:
df = df.dropna()

In [ ]:
# Changing the name of below two columns for ease to access
df.rename(columns = {'Industry Classification Tag':'Label'}, inplace = True) 
df.rename(columns = {'Business Description':'Busn_Desc'}, inplace = True) 

In [ ]:
df.head(5)

,Busn_Desc,Label
0,"Adsouth Partners, Inc. provides advertising ag...",Advertising
1,"Artec Global Media, Inc., formerly Artec Consu...",Advertising
2,Betawave Corporation provides online marketing...,Advertising
3,Boston Omaha Corporation is engaged in the bus...,Advertising
4,"Bright Mountain Media, Inc. is a digital media...",Advertising


Perform Label Encoding since it contains Categorical Features

In [ ]:
# Import label encoder 
from sklearn import preprocessing 
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
df['Label']= label_encoder.fit_transform(df['Label']) 
  
df['Label'].unique() 

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34,
       35, 36, 37, 31, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61])

In [ ]:
df.head()

,Busn_Desc,Label
0,"Adsouth Partners, Inc. provides advertising ag...",0
1,"Artec Global Media, Inc., formerly Artec Consu...",0
2,Betawave Corporation provides online marketing...,0
3,Boston Omaha Corporation is engaged in the bus...,0
4,"Bright Mountain Media, Inc. is a digital media...",0


In [ ]:
msg = df.copy()
msg.reset_index(inplace=True)
msg.head(10)

,index,Busn_Desc,Label
0,0,"Adsouth Partners, Inc. provides advertising ag...",0
1,1,"Artec Global Media, Inc., formerly Artec Consu...",0
2,2,Betawave Corporation provides online marketing...,0
3,3,Boston Omaha Corporation is engaged in the bus...,0
4,4,"Bright Mountain Media, Inc. is a digital media...",0
5,5,"Cardlytics, Inc. is engaged in developing a pu...",0
6,6,"CMG Holdings Group, Inc. is a holding company....",0
7,7,"comScore, Inc. is a cross-platform measurement...",0
8,8,"Connected Media Technologies, Inc. is a sales ...",0
9,9,"Credit One Financial, Inc., through its subsid...",0


In [ ]:
len(msg)

6022

In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0,len(msg)):
  review = re.sub('[^a-zA-Z]', ' ', msg['Busn_Desc'][i])
  review = review.lower()
  review = review.split()
  ps = PorterStemmer()  
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
corpus[3]

'boston omaha corpor engag busi variou sector includ advertis insur real estat compani oper includ ownership multipl billboard alabama florida georgia west virginia wisconsin equiti method invest sever real estat real estat servic compani compani insur oper conduct subsidiari gener indemn group llc compani oper includ billboard advertis busi insur busi special sureti bond compani subsidiari includ link media hold llc gener indemn direct insur servic llc'

In [ ]:
## Applying Countvectorizer
# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=1500)
X = cv.fit_transform(corpus).toarray()

In [ ]:
X.shape

(6022, 1500)

In [ ]:
y = msg['Label']
y.shape

(6022,)

In [ ]:
len(X[0])

1500

In [ ]:
## Divide the dataset into Train and Test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
cv.get_feature_names()[:20]

['ab',
 'abil',
 'acceler',
 'accept',
 'access',
 'accessori',
 'accommod',
 'account',
 'acid',
 'acquir',
 'acquisit',
 'acr',
 'acreag',
 'across',
 'act',
 'action',
 'activ',
 'acut',
 'ad',
 'adapt']

In [ ]:
cv.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.int64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 1.0,
 'max_features': 1500,
 'min_df': 1,
 'ngram_range': (1, 1),
 'preprocessor': None,
 'stop_words': None,
 'strip_accents': None,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': None,
 'vocabulary': None}

Implement classifiers and compute Accuracy, F1 Score and AUC

In [ ]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
y_pred = classifier.predict(X_test)
from sklearn.metrics import confusion_matrix,accuracy_score,f1_score
cm = confusion_matrix(y_test,y_pred)
#print(cm)
print ('Accuracy:',accuracy_score(y_test, y_pred))
print ('F1 score:', f1_score(y_test, y_pred,average='weighted'))

from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred,pos_label=61)
print ('AUC score:',metrics.auc(fpr, tpr))

Accuracy: 0.43568464730290457
F1 score: 0.42663425057629834
AUC score: 0.4511800334168755


In [ ]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
y_pred = mnb.predict(X_test)
cm = confusion_matrix(y_test,y_pred)
#print(cm)
print ('Accuracy:',accuracy_score(y_test, y_pred))
print ('F1 score:', f1_score(y_test, y_pred,average='weighted'))

fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred,pos_label=61)
print ('AUC score:',metrics.auc(fpr, tpr))

Accuracy: 0.6672199170124481
F1 score: 0.6599388246612796
AUC score: 0.5982664995822891


In [ ]:
from sklearn.linear_model import PassiveAggressiveClassifier
pac = PassiveAggressiveClassifier()
pac.fit(X_train, y_train)

PassiveAggressiveClassifier(C=1.0, average=False, class_weight=None,
                            early_stopping=False, fit_intercept=True,
                            loss='hinge', max_iter=1000, n_iter_no_change=5,
                            n_jobs=None, random_state=None, shuffle=True,
                            tol=0.001, validation_fraction=0.1, verbose=0,
                            warm_start=False)

In [ ]:
y_pred = pac.predict(X_test)
cm = confusion_matrix(y_test,y_pred)
#print(cm)
print ('Accuracy:',accuracy_score(y_test, y_pred))
print ('F1 score:', f1_score(y_test, y_pred,average='weighted'))

fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred,pos_label=61)
print ('AUC score:',metrics.auc(fpr, tpr))

Accuracy: 0.6116182572614108
F1 score: 0.6071608302808356
AUC score: 0.7199770258980785


In [ ]:
from sklearn.linear_model import LogisticRegression
log = LogisticRegression(random_state = 0)
log.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
y_pred = log.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
#print(cm)
print ('Accuracy:',accuracy_score(y_test, y_pred))
print ('F1 score:', f1_score(y_test, y_pred,average='weighted'))

fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred,pos_label=61)
print ('AUC score:',metrics.auc(fpr, tpr))

Accuracy: 0.645643153526971
F1 score: 0.6415982456951899
AUC score: 0.6920426065162907


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
y_pred = knn.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
#print(cm)
print ('Accuracy:',accuracy_score(y_test, y_pred))
print ('F1 score:', f1_score(y_test, y_pred,average='weighted'))

fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred,pos_label=61)
print ('AUC score:',metrics.auc(fpr, tpr))

Accuracy: 0.45726141078838173
F1 score: 0.44501272141021425
AUC score: 0.42387218045112784


In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
dt.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [ ]:
y_pred = dt.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
#print(cm)
print ('Accuracy:',accuracy_score(y_test, y_pred))
print ('F1 score:', f1_score(y_test, y_pred,average='weighted'))

fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred,pos_label=61)
print ('AUC score:',metrics.auc(fpr, tpr))

Accuracy: 0.3369294605809129
F1 score: 0.3379111166145514
AUC score: 0.556234335839599


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [ ]:
y_pred = rf.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
#print(cm)
print ('Accuracy:',accuracy_score(y_test, y_pred))
print ('F1 score:', f1_score(y_test, y_pred,average='weighted'))

fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred,pos_label=61)
print ('AUC score:',metrics.auc(fpr, tpr))

Accuracy: 0.3817427385892116
F1 score: 0.338576631763304
AUC score: 0.4137426900584795
